In [68]:
# config
import __confing
__confing.change_to_root_folder()

%load_ext autoreload
%autoreload 2

Current directory: c:\Users\Leo\OneDrive\Escritorio\Final Proyect
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
# imports
import pandas as pd
import optuna

from src import value_filler as vs
from src import feature_selection as fs
from src import data_procces as dp
from src import evaluation as ev

from sklearn.metrics import accuracy_score, classification_report, mean_squared_error,roc_auc_score,confusion_matrix

In [91]:
def get_performance(model, X_train, X_test, y_train, y_test):
    y_pred = model.predict(X_test)
    roc_auc_train = roc_auc_score(y_train, model.predict(X_train))
    # Validation ROC AUC Score
    acc = accuracy_score(y_test, y_pred)
    roc_auc_val = roc_auc_score(y_test, y_pred)
    print( f"Model Accuracy: {acc}",
            f"Train roc_auc: {roc_auc_train}", 
            f"Test roc_auc: {roc_auc_val}",
            sep='\n', end='\n\n')
    print(confusion_matrix(y_test, y_pred),
    classification_report(y_test, y_pred),
    sep='\n'
)

# `Data process`

In [105]:
# dataframe
df = pd.read_csv(r'data\waves\all_waves.csv')
target = 'rahosp1y'

In [106]:
# drop
df = df.drop(['rahspnit1y', 'wave', 'rahhidnp'], axis=1)

df = df.dropna(subset=[target])

porcentaje_nulos = df.isnull().sum(axis=1) / len(df.columns)
df = df[(df['rahosp1y'] != 0) | (porcentaje_nulos <= 0.4)]


In [107]:
# filling
df = fs.fast_fill(df)

In [115]:
# filtering
l1 = fs.get_corr_columns(df, target, True)
l2 = fs.get_corr_columns(df, target, False)
l3 = l1+l2

df_f = df[l3]
df_f[target] = df[target]


C:\Users\Leo\AppData\Local\Temp\ipykernel_14896\1131890659.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f[target] = df[target]


In [133]:
# split --
y = df_f['rahosp1y']
X = df_f.drop('rahosp1y', axis=1)

X_train, X_test, y_train, y_test = dp.split_data(
    X, y, test_size=0.2, random_state=40)


In [134]:
# resample quitado*
X_train, y_train = dp.apply_resample(X_train, y_train,v=1.6)
X_test, y_test = dp.apply_resample(X_test, y_test,v=1)

In [135]:
# smote
X_train, y_train = dp.apply_smote(X_train, y_train)

In [136]:
# scaler
X_train, X_test = dp.apply_standard_scaler(X_train, X_test)


In [137]:
# power transform
X_train, X_test = dp.apply_power_transform(X_train, X_test)


In [ ]:
raise ValueError("stoped")

# `Models`

## AdaBoost


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
raise ValueError("comment this line to run the cell")

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.1)
    algorithm = trial.suggest_categorical('algorithm', ['SAMME', 'SAMME.R'])
    random_state = trial.suggest_int('random_state', 0, 100)

    base_estimator = DecisionTreeClassifier(
        max_depth=1, random_state=random_state)
    model = AdaBoostClassifier(
        base_estimator=base_estimator,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        algorithm=algorithm,
        random_state=random_state
    )
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=False)

In [ ]:
# Imprimir los mejores resultados
print('Mejor precisión encontrada:', study.best_value)
print('Mejores hiperparámetros encontrados:', study.best_params)
adaboost_params = study.best_params


Mejor precisión encontrada: 0.9101548812110168
Mejores hiperparámetros encontrados: {'n_estimators': 123, 'learning_rate': 0.024457369594744295, 'algorithm': 'SAMME', 'random_state': 88}


In [ ]:
adaboost_params ={
    'n_estimators': 123, 
    'learning_rate': 0.024457369594744295, 
    'algorithm': 'SAMME', 
    'random_state': 88
}

## CatBoost


In [ ]:
from catboost import CatBoostClassifier

In [ ]:
raise ValueError("comment this line to run the cell")

def objective(trial):
    # Definir los espacios de búsqueda para los hiperparámetros de CatBoost
    params = {
        'iterations': trial.suggest_int('iterations', 50, 200),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.00001, 0.1),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 0.1, 10.0),
        'random_seed': trial.suggest_int('random_seed', 1, 100),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 1.0),
        'loss_function': trial.suggest_categorical('loss_function', ['Logloss', 'CrossEntropy']),
        'eval_metric': trial.suggest_categorical('eval_metric', ['Accuracy', 'F1', 'AUC']),
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 5, 20),
    }

    # Crear el clasificador CatBoost con los hiperparámetros sugeridos
    model = CatBoostClassifier(**params)
    # Entrenar el modelo
    model.fit(X_train, y_train, verbose=False)
    # Calcular las predicciones en el conjunto de prueba
    y_pred = model.predict(X_test)
    # Calcular la precisión del modelo
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy


# Crear el estudio de Optuna y optimizar los hiperparámetros
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=False)

# Obtener los mejores resultados
best_value = study.best_value
best_params = study.best_params

In [ ]:
# Imprimir los mejores resultados
print('Mejor precisión encontrada:', study.best_value)
print('Mejores parámetros encontrados:', study.best_params)
catboost_params = study.best_params


Mejor precisión encontrada: 0.9044081575443269
Mejores parámetros encontrados: {'iterations': 187, 'learning_rate': 0.040676323292287556, 'depth': 7, 'l2_leaf_reg': 2.9831156146703663, 'random_seed': 66, 'bootstrap_type': 'Bernoulli', 'subsample': 0.3795675559951529, 'colsample_bylevel': 0.8542347504139569, 'loss_function': 'Logloss', 'eval_metric': 'F1', 'early_stopping_rounds': 14}


In [ ]:
catboost_params = {
    'iterations': 187, 
    'learning_rate': 0.040676323292287556, 
    'depth': 7, 
    'l2_leaf_reg': 2.9831156146703663, 
    'random_seed': 66, 
    'bootstrap_type': 'Bernoulli',
    'subsample': 0.3795675559951529, 
    'colsample_bylevel': 0.8542347504139569, 
    'loss_function': 'Logloss', 
    'eval_metric': 'F1', 
    'early_stopping_rounds': 14
}
model = CatBoostClassifier(**catboost_params)
model.fit(X_train, y_train, verbose=False)
y_pred = model.predict(X_test)

get_performance(model, X_train, X_test, y_train, y_test)

# Model Accuracy: 0.9044081575443269
# Train roc_auc: 0.8338428430642987
# Test roc_auc: 0.7009613815616501

Model Accuracy: 0.7270255573681349
Train roc_auc: 0.7453974895397489
Test roc_auc: 0.7270255573681349

[[1687  152]
 [ 852  987]]
              precision    recall  f1-score   support

         0.0       0.66      0.92      0.77      1839
         1.0       0.87      0.54      0.66      1839

    accuracy                           0.73      3678
   macro avg       0.77      0.73      0.72      3678
weighted avg       0.77      0.73      0.72      3678



## LightGBM


In [ ]:
import lightgbm as lgb

In [ ]:
raise ValueError("comment this line to run the cell")

def objective(trial):
    params = {
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart', 'rf']),
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'max_depth': trial.suggest_int('max_depth', -1, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.00001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 100000, 300000),
        'objective': 'binary',  
        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced']),
        'min_split_gain': trial.suggest_float('min_split_gain', 0., 1.),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-3, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 30),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'subsample_freq': trial.suggest_int('subsample_freq', 0, 5),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0., 1.),
        'reg_lambda': trial.suggest_float('reg_lambda', 0., 1.),
        'random_state': trial.suggest_int('random_state', 1, 100),
        'importance_type': trial.suggest_categorical('importance_type', ['split', 'gain']),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),  
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),  
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Return accuracy for maximization
    return accuracy


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=False)

In [ ]:
# Imprimir los mejores resultados
print('Mejor precisión encontrada:', study.best_value)
print('Mejores parámetros encontrados:', study.best_params)

lgbm_params = study.best_params

Mejor precisión encontrada: 0.9101548812110168
Mejores parámetros encontrados: {'boosting_type': 'rf', 'num_leaves': 61, 'max_depth': 1, 'learning_rate': 0.01816560652901298, 'n_estimators': 87, 'subsample_for_bin': 290089, 'class_weight': 'balanced', 'min_split_gain': 0.4341050061211783, 'min_child_weight': 0.30754031398986437, 'min_child_samples': 11, 'subsample': 0.9812553236546777, 'subsample_freq': 4, 'colsample_bytree': 0.8148632932127071, 'reg_alpha': 0.9556694136778114, 'reg_lambda': 0.49691043770864046, 'random_state': 53, 'importance_type': 'split', 'bagging_fraction': 0.3099214263868336, 'bagging_freq': 7}


In [ ]:
lgbm_params = {
    'boosting_type': 'rf', 
    'num_leaves': 61, 
    'max_depth': 1, 
    'learning_rate': 0.01816560652901298, 
    'n_estimators': 87, 
    'subsample_for_bin': 290089, 
    'class_weight': 'balanced', 
    'min_split_gain': 0.4341050061211783, 
    'min_child_weight': 0.30754031398986437, 
    'min_child_samples': 11, 
    'subsample': 0.9812553236546777, 
    'subsample_freq': 4, 
    'colsample_bytree': 0.8148632932127071, 
    'reg_alpha': 0.9556694136778114, 
    'reg_lambda': 0.49691043770864046, 
    'random_state': 53, 
    'importance_type': 'split', 
    'bagging_fraction': 0.3099214263868336, 
    'bagging_freq': 7
} # 95 auc

model = lgb.LGBMClassifier(**lgbm_params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
get_performance(model, X_train, X_test, y_train, y_test)

[LightGBM] [Warning] bagging_fraction is set=0.3099214263868336, subsample=0.9812553236546777 will be ignored. Current value: bagging_fraction=0.3099214263868336
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=4 will be ignored. Current value: bagging_freq=7
Model Accuracy: 0.6514410005437737
Train roc_auc: 0.6577928870292887
Test roc_auc: 0.6514410005437739

[[1839    0]
 [1282  557]]
              precision    recall  f1-score   support

         0.0       0.59      1.00      0.74      1839
         1.0       1.00      0.30      0.46      1839

    accuracy                           0.65      3678
   macro avg       0.79      0.65      0.60      3678
weighted avg       0.79      0.65      0.60      3678



## XGBoost


In [ ]:
from xgboost import XGBClassifier

In [ ]:
raise ValueError("comment this line to run the cell")

def objective(trial):
    params = {
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'eta': trial.suggest_loguniform('eta', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.0001, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'eval_metric': 'mlogloss',
        'use_label_encoder': False
    }
    # Fit the model
    optuna_model = XGBClassifier(**params)
    optuna_model.fit(X_train, y_train)
    # Make predictions
    y_pred = optuna_model.predict(X_test)
    # Evaluate predictions
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy


# Create the Optuna study and optimize the parameters
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=False)

In [ ]:
# Imprimir los mejores resultados
print('Mejor precisión encontrada:', study.best_value)
print('Mejores parámetros encontrados:', study.best_params)

xgboost_params = study.best_params

Mejor precisión encontrada: 0.9095241432475997
Mejores parámetros encontrados: {'booster': 'gbtree', 'eta': 0.012093142041979083, 'max_depth': 6, 'subsample': 0.8874936946322917, 'learning_rate': 0.14552872398461472, 'n_estimators': 245, 'min_child_weight': 5, 'gamma': 5.264572425726908e-05, 'colsample_bytree': 0.3424548012820728, 'reg_alpha': 1.3334029646882244e-06, 'reg_lambda': 0.0006323060575623945}


In [139]:
xgboost_params ={
    'booster': 'gbtree', 
    'eta': 0.012093142041979083, 
    'max_depth': 6, 
    'subsample': 0.8874936946322917, 
    'learning_rate': 0.14552872398461472, 
    'n_estimators': 245, 
    'min_child_weight': 5, 
    'gamma': 5.264572425726908e-05, 
    'colsample_bytree': 0.3424548012820728, 
    'reg_alpha': 1.3334029646882244e-06, 
    'reg_lambda': 0.0006323060575623945
} # 86 auc

model = XGBClassifier(**xgboost_params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
get_performance(model, X_train, X_test, y_train, y_test)

Model Accuracy: 0.7183251767264818
Train roc_auc: 0.7980622131565527
Test roc_auc: 0.7183251767264818

[[1686  153]
 [ 883  956]]
              precision    recall  f1-score   support

         0.0       0.66      0.92      0.76      1839
         1.0       0.86      0.52      0.65      1839

    accuracy                           0.72      3678
   macro avg       0.76      0.72      0.71      3678
weighted avg       0.76      0.72      0.71      3678



## Stochastic Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from joblib import parallel_backend

In [ ]:
def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'random_state': 42,
    }
    
    model = GradientBoostingClassifier(**params)
    with parallel_backend('multiprocessing'):
        model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=False)

In [ ]:
print('Mejor precisión encontrada:', study.best_value)
print('Mejores parámetros encontrados:', study.best_params)
stochastic_params = study.best_params

## Voting

In [ ]:
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [ ]:
def objective(trial):
    catboost_params = {
        'iterations': trial.suggest_int('catboost_iterations', 50, 200),
        'learning_rate': trial.suggest_loguniform('catboost_learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('catboost_depth', 3, 10),
        # Otros parámetros de CatBoost
    }
    lgbm_params = {
        'learning_rate': trial.suggest_loguniform('lgbm_learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('lgbm_n_estimators', 50, 200),
        'max_depth': trial.suggest_int('lgbm_max_depth', 1, 10),
        # Otros parámetros de LightGBM
    }
    xgboost_params = {
        'learning_rate': trial.suggest_loguniform('xgboost_learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('xgboost_n_estimators', 50, 200),
        'max_depth': trial.suggest_int('xgboost_max_depth', 1, 10),
        # Otros parámetros de XGBoost
    }
    
    catboost_model = CatBoostClassifier(**catboost_params)
    lgbm_model = LGBMClassifier(**lgbm_params)
    xgboost_model = XGBClassifier(**xgboost_params)
    
    estimators = [
        ('catboost', catboost_model),
        ('lgbm', lgbm_model),
        ('xgboost', xgboost_model)
    ]
    
    voting = VotingClassifier(estimators, voting='soft')
    voting.fit(X_train, y_train)
    y_pred = voting.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Retornar la precisión para maximizar
    return accuracy


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=False)

best_params = study.best_params
catboost_params = {k[10:]: v for k,v in best_params.items() if k.startswith('catboost_')}
lgbm_params = {k[5:]: v for k,v in best_params.items() if k.startswith('lgbm_')}
xgboost_params = {k[8:]: v for k,v in best_params.items() if k.startswith('xgboost_')}

In [ ]:
print("Best CatBoost params:", catboost_params)
print("Best LightGBM params:", lgbm_params)
print("Best XGBoost params:", xgboost_params)
